In [37]:
# Import Libraries

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import preprocessing
from sklearn import metrics
import random
import sys
sys.path.append('../')


In [38]:
# Import data from file
df = pd.read_csv(os.getcwd() + '/agaricus-lepiota.data', sep=',', header=None,
             error_bad_lines=False, warn_bad_lines=True, low_memory=False)

# Set pandas to output all of the columns in output
df.columns = ['class','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment',
         'gill-spacing','gill-size','gill-color','stalk-shape','stalk-root',
         'stalk-surf-above-ring','stalk-surf-below-ring','stalk-color-above-ring','stalk-color-below-ring',
         'veil-type','veil-color','ring-number','ring-type','spore-color','population','habitat']

# Split context from label
X = pd.DataFrame(df, columns=df.columns[1:len(df.columns)], index=df.index)
# Put the class values (0th column) into Y
Y = df['class']

# Transform labels into one-hot encoded array
le = preprocessing.LabelEncoder()
le.fit(Y)
y = le.transform(Y)

# Temporary variable to avoid error 
x_tmp = pd.DataFrame(X,columns=[X.columns[0]])

# Encode each feature column and add it to x_train 
for colname in X.columns:
    le.fit(X[colname])
    #print(colname, le.classes_)
    x_tmp[colname] = le.transform(X[colname])

# Produce mushroom array: 8124 mushrooms, each with 117 one-hot encoded features
oh = preprocessing.OneHotEncoder(categorical_features='all')
oh.fit(x_tmp)
x = oh.transform(x_tmp).toarray()

/Users/lorcandelaney/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning:

The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.

/Users/lorcandelaney/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:380: DeprecationWarning:

The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. The passed value of 'all' is the default and can simply be removed.



In [39]:
print(len(x))

8124


In [40]:
# REWARD FUNCTION

def get_reward(eaten, edible):
    # REWARDS FOR AGENT
    #  Eat poisonous mushroom
    if not eaten:
        return 0
    if eaten and edible:
        return 5
    elif eaten and not edible:
        return (5 if np.random.rand() > 0.5 else -35)

def oracle_reward(edible):
    return 5*edible   

In [41]:
def init_buffer():
    bufferX, bufferY = [], []
    for i in np.random.choice(range(len(x)), 4096):
        eat = np.random.rand()>0.5
        bufferX.append(np.concatenate((x[i], [1, 0] if eat else [0, 1])))
        bufferY.append(get_reward(eat, y[i]))
    return bufferX, bufferY

In [42]:
import torch.nn as nn
import torch.nn.functional as F
import sys
from torch.autograd import Variable

Var = lambda x, dtype=torch.FloatTensor: Variable(torch.from_numpy(x).type(dtype))

In [43]:
class MushroomNet():
    def __init__(self, label='MushNet', n_weight_sampling = 2):
        self.n_weight_sampling = n_weight_sampling
        self.label = label
        self.epsilon = 0
        self.net = None
        self.bufferX, self.bufferY = init_buffer()
        self.loss, self.optimizer = None, None
        self.cum_regrets = [0]
    
    def expected_rewards(self, context):
        k = self.n_weight_sampling
        c_eat = Var(np.concatenate((context, [1, 0])))
        c_reject = Var(np.concatenate((context, [0, 1])))
        with torch.no_grad():
            r_eat = np.mean([self.net.forward(c_eat).numpy().reshape(1)[0] for _ in range(k)])
            r_reject = np.mean([self.net.forward(c_reject).numpy().reshape(1)[0] for _ in range(k)])
        return r_reject, r_eat

    def try_(self, mushroom):
        context, edible = x[mushroom], y[mushroom]
        r_reject, r_eat = self.expected_rewards(context)
        eaten = r_eat > r_reject
        if np.random.rand()<self.epsilon:
            eaten = (np.random.rand()<.5)
        reward = get_reward(eaten, edible)
        action = [1, 0] if eaten else [0, 1]
        self.bufferX.append(np.concatenate((context, action)))
        self.bufferY.append(reward)
        rg = oracle_reward(edible) - reward
        self.cum_regrets.append(self.cum_regrets[-1]+rg)
    
    def update(self, mushroom):
        self.try_(mushroom)
        bX = Var(np.array(self.bufferX[-4096:]))
        bY = Var(np.array(self.bufferY[-4096:]))
        for idx in np.split(np.random.permutation(range(4096)), 64):
            #self.net.train()
            self.net.zero_grad()
            self.loss(bX[idx], bY[idx]).backward()
            self.optimizer.step()

In [44]:
class EpsGreedyMlp(MushroomNet):
    def __init__(self, epsilon=0, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        self.net = nn.Sequential(
        nn.Linear(x.shape[1]+2, 100), nn.ReLU(),
        nn.Linear(100, 100), nn.ReLU(),
        nn.Linear(100, 1))
        self.bufferX, self.bufferY = init_buffer()
        self.optimizer = optim.Adam(self.net.parameters())
        self.mse = nn.MSELoss()
        self.loss = lambda data, target: self.mse(self.net.forward(data), target)

In [45]:
mushroom_nets = {
                 'e0':EpsGreedyMlp(epsilon=0, label = 'Greedy'),
                 'e1':EpsGreedyMlp(epsilon=0.01, label = '1% Greedy'),
                 'e5':EpsGreedyMlp(epsilon=0.05, label = '5% Greedy')}

In [46]:
import plotly.graph_objs as go
class Fig(go.FigureWidget):
    def __init__(self):
        super().__init__([go.Scatter(name = net.label, x=[], y=[]) 
                          for i, net in mushroom_nets.items()])
        self.layout.title = 'Cumulative Regrets'
    
    def update(self, dic):
        for j, (lb, y) in enumerate(dic.items()):
            self.data[j].x = list(range(len(y)))
            self.data[j].y = y
    
    def save(self, file_name = 'mushroom_regrets.csv'):
        dic = {fd.name: fd.y for fd in self.data}
        pd.DataFrame.from_dict(dic).to_csv(file_name)
        
        
fig = Fig()
fig  

Fig({
    'data': [{'name': 'Greedy', 'type': 'scatter', 'uid': '818eadbc-93a8-46b0-9a84-4f62ace14888', 'x': [], 'y': []},
             {'name': '1% Greedy', 'type': 'scatter', 'uid': 'd5e6e298-f551-4efa-a081-ef135ac7ab81', 'x': [], 'y': []},
             {'name': '5% Greedy', 'type': 'scatter', 'uid': '68d3b72b-6a9d-4c5b-b8d2-7b734ebc8a85', 'x': [], 'y': []}],
    'layout': {'title': {'text': 'Cumulative Regrets'}}
})

In [47]:
import plotly.offline as py
py.iplot(fig)

In [48]:
NB_STEPS = 5000

offset = len(fig.data[0].x)
for _ in range(offset, offset+NB_STEPS):
    mushroom = np.random.randint(len(x))
    for j, (key, net) in enumerate(mushroom_nets.items()):
        net.update(mushroom)
    fig.update({net.label: net.cum_regrets for lb, net in mushroom_nets.items()})